In [1]:
import torch as th
import pytorch_lightning as pl
from utils.dataloader_pl import SWEDataModule

dataset = SWEDataModule(partial=0.01)

In [ ]:
from models_lightning.ae import main
from importlib import reload  
main = reload(main)

model = main.deepSWE(nf=4, in_chan=4)

trainer = pl.Trainer()
trainer.fit(model, dataset)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


[~] Loading from disk...
[x] 25 areas found
. . . . . . . 

KeyboardInterrupt: 